### CLANに流れていたコマンドを確認する

In [47]:
import datetime
import csv

#### 文字列の整形 (佐藤さんのコード)

In [55]:
# from Sato-san's ana-CLAN_MONITOR.py
def cid2cname(id):
    try:
        ret = dict1[id]
    except:
        ret = "Unknown Command"
    return ret

def cid2jpdsc(id):
    try:
        ret = dict2[id]
    except:
        ret = ""
    return ret

def cid2dname(id):
    did = id[:3]
    try:
        ret = dict3[did]
    except:
        ret = "Undeined Device"
    return ret


# read command list
with open('CMDNAME.LST.tsv', 'r') as f:
    f1dat = csv.reader(f, delimiter = '\t')
    dict1 = {}
    for row in f1dat:
        dict1[row[0]] = row[1]

# read command list
with open('ID-JPDIST.tsv', 'r') as f:
    f2dat = csv.reader(f, delimiter = '\t')
    dict2 = {}
    for row in f2dat:
    	dict2[row[0]] = row[1]

# read command list
with open('CID-DNAME.tsv', 'r') as f:
    f3dat = csv.reader(f, delimiter = '\t')
    dict3 = {}
    for row in f3dat:
    	dict3[row[0]] = row[1]

In [77]:
# from Sato-san's ana-CLAN_MONITOR.py
def formatMessage(mt, dst, src, pid, len, body, hst):
	#ret = mt + d + dst + d + src + d + pid + d + mst + d + len
    d = ' '
    ret = str(hst)[:21] + d + mt + d + src + ' > ' + dst
#	ret = mt + d + dst + d + src + d + pid + d + str(hst) + d + len
	# command body analysis
    if mt in {"CC", "CD"}:
        cdn = body[:6]		# command demand number
        cid = body[6:12]	# command id
        prm = body[12:]		# parameter
		#cnm = dict[cid]		# command name
        cnm = cid2cname(cid)		# command name
        dnm = cid2dname(cid)
		#cjp = cid2jpdsc(cid)		# command discription
        ret = ret + d + '0x' + cdn + d + '0x' + cid + d + dnm + d + cnm + d + prm
    elif mt == "CA":
        cdn = body[:6]		# command demand number
        cid = body[6:12]	# command id
        can = body[12:18]	# command acceptance number
        res = body[18:20]	# acceptance result
        inf = body[20:6]	# acceptance infomation
        cnm = cid2cname(cid)		# command name
        dnm = cid2dname(cid)
        cjp = cid2jpdsc(cid)		# command discription
		#cnm = dict[cid]		# command name
		#ret = ret + d + '0x' + cdn + d + '0x' + cid + d + cnm + d + '0x' + can + d + res + d + inf
        ret = ret + d + '0x' + cdn + d + '0x' + cid + d + dnm + d + cnm + d + res + d + inf
    elif mt == "CE":
        cdn = body[:6]		# command demand number
        cid = body[6:12]	# command id
        can = body[12:18]	# command acceptance number
        res = body[18:28] 	# execution result
        prm = body[28:]		# executon parameter
        cnm = cid2cname(cid)		# command name
        dnm = cid2dname(cid)
        cjp = cid2jpdsc(cid)		# command discription
		#ret = ret + d + '0x' + cdn + d + '0x' + cid + d + cnm + d + '0x' + can + d + res + d + prm
        ret = ret + d + '0x' + cdn + d + '0x' + cid + d + dnm + d + cnm + d + res + d + prm
    elif mt == "CR":
        cdn = body[:6]		# command demand number
        cid = body[6:12]	# command id
        gid = body[12:14]	# group id
        mcr = body[14:30]	# macro name 
        exn = body[30:33]	# execution number
        pid = body[33:39]	# primitive command id
        res = body[39:49]	# execution result
        prm = body[49:]		# execution result parameter
        cnm = cid2cname(cid)		# command name
        dnm = cid2dname(cid)
        cjp = cid2jpdsc(cid)		# command discription
        ret = ret + d + '0x' + cdn + d + '0x' + cid + d + dnm + d + cnm + d + gid + d + mcr + d + exn + d + pid + d + res + d + prm
    elif mt == "CM":
        cat = body[:1]		# category
        if cat == '1':
            cat = "CAUTION"
        elif cat == '2':
            cat = "ERROR  "
        else:
            cat = "OTHERS "
        mcd = body[1:7]		# message code
        if   mcd == '000001':
            mcd = "OBS-TSC Communication Error"
        elif mcd == '000002':
            mcd = "TWS1-TSC Communication Error"
        elif mcd == '000003':
            mcd = "TWS2-TSC Communication Error"
        elif mcd == '000004':
            mcd = "TWS3-TSC Communication Error"
        elif mcd == '000005':
            mcd = "TSC-MLP1 Communication Error"
        elif mcd == '000006':
            mcd = "TSC-MLP2 Communication Error"
        elif mcd == '000007':
            mcd = "TSC-MLP3 Communication Error"
        else:
            pass 	
        med = body[7:8]		# message enable/disable
        if   med == '0':
            med = "DIS"
        elif med == '1':
            med = "ENA"
        else:
            pass
        msg = body[8:]		# message
        ret = ret + d + d + mcd + d + d + cat + d + msg
    elif mt == "CP":
        cdn = body[:6]		# command demand number
        cid = body[6:12]	# command id
        can = body[12:18]	# command acceptance number
        ukn = body[18:25]	# unknown
        otr = body[25:]		# other
        cnm = cid2cname(cid)		# command name
        dnm = cid2dname(cid)
        cjp = cid2jpdsc(cid)		# command discription
		#ret = ret + d + '0x' + cdn + d + '0x' + cid + d + cnm + d + '0x' + can + d + ukn + d + otr
        ret = ret + d + '0x' + cdn + d + '0x' + cid + d + dnm + d + cnm + d + ukn + d + otr
    else:
        ret = '? ' + ret + d + body
    return(ret)


#### 調査対象の時刻レンジの指定

In [120]:
startDateTime = datetime.datetime(2024, 3, 20, 4, 55, 0)

durationInSec = 1200

stopDateTime = startDateTime + datetime.timedelta(seconds=durationInSec)
# a log file covers the time range of HST 14:00: 14:00+1d

if startDateTime.strftime('%p') == 'PM':
    logDay = startDateTime  + datetime.timedelta(days=1)
else:
    # AM
    logDay = startDateTime
    
filePath = f'/tscbin/tsc/{logDay.year}/CLAN_MONITOR.{logDay.year}{logDay.month:02d}{logDay.day:02d}.LOG'
print(filePath)
with open(filePath) as f:
    lines = [s.rstrip() for s in f.readlines()] # chop the last CR

for line in lines:
    msg = line[24:]
    mst = msg[14:29]
    date = datetime.date(int(mst[:4]), int(mst[4:6]), int(mst[6:8]))
    time = datetime.time(int(mst[8:10]), int(mst[10:12]), int(mst[12:14]), int(mst[14:])*100000)
    ut  = datetime.datetime.combine(date, time)
    hst = ut - datetime.timedelta(hours=10)
    
    if (hst < startDateTime) or (hst > stopDateTime):
        continue

    time = lines[:15]
    mt = msg[0:2]
    dst = msg[2:6]
    src = msg[6:10]
    pid = msg[10:14]
    len = msg[29:33]
    body = msg[33:]

    hst = hst.strftime('%Y-%m-%d %H:%M:%S.%f')
    result = formatMessage(mt, dst, src, pid, len, body, hst)
    print(result)

/tscbin/tsc/2024/CLAN_MONITOR.20240320.LOG
2024-03-20 04:57:40.8 CD OCS% > TSC% 0x000566 0x906012 AG Set AG Star1 REF Position 23059 20115
2024-03-20 04:57:40.7 CA TSC% > OCS% 0x000566 0x906012 AG Set AG Star1 REF Position OK 
2024-03-20 04:57:41.0 CE TSC% > OCS% 0x000566 0x906012 AG Set AG Star1 REF Position COMPLETE%% 0006202403201457410
2024-03-20 05:00:01.9 CD TWS4 > TSC% 0x001B3B 0x1A1011 TSC Send Status Data 
2024-03-20 05:00:01.8 CA TSC% > TWS4 0x001B3B 0x1A1011 TSC Send Status Data OK 
2024-03-20 05:00:01.8 CE TSC% > TWS4 0x001B3B 0x1A1011 TSC Send Status Data COMPLETE%% 
2024-03-20 05:01:09.8 CD OCS% > TSC% 0x000567 0x906012 AG Set AG Star1 REF Position 23123 18706
2024-03-20 05:01:09.7 CA TSC% > OCS% 0x000567 0x906012 AG Set AG Star1 REF Position OK 
2024-03-20 05:01:10.0 CE TSC% > OCS% 0x000567 0x906012 AG Set AG Star1 REF Position COMPLETE%% 0006202403201501100
2024-03-20 05:01:19.8 CD OCS% > TSC% 0x000568 0x906012 AG Set AG Star1 REF Position 23182 17406
2024-03-20 05:01:1

In [5]:
import datetime
from telDav import ClanMonitor

clanMonitor = ClanMonitor.ClanMonitor(startDatetime=datetime.datetime(2024, 3, 20, 4, 55, 0),
                                      stopDatetime=datetime.datetime(2024, 3, 20, 5, 15, 0),
                                      keyWords=[""])

lines = clanMonitor.run()
for l in lines:
    print(l)

/tscbin/tsc/2024/CLAN_MONITOR.20240320.LOG
2024-03-20 04:57:40.8 CD OCS% > TSC% 0x000566 0x906012 AG Set AG Star1 REF Position 23059 20115
2024-03-20 04:57:40.7 CA TSC% > OCS% 0x000566 0x906012 AG Set AG Star1 REF Position OK 
2024-03-20 04:57:41.0 CE TSC% > OCS% 0x000566 0x906012 AG Set AG Star1 REF Position COMPLETE%% 0006202403201457410
2024-03-20 05:00:01.9 CD TWS4 > TSC% 0x001B3B 0x1A1011 TSC Send Status Data 
2024-03-20 05:00:01.8 CA TSC% > TWS4 0x001B3B 0x1A1011 TSC Send Status Data OK 
2024-03-20 05:00:01.8 CE TSC% > TWS4 0x001B3B 0x1A1011 TSC Send Status Data COMPLETE%% 
2024-03-20 05:01:09.8 CD OCS% > TSC% 0x000567 0x906012 AG Set AG Star1 REF Position 23123 18706
2024-03-20 05:01:09.7 CA TSC% > OCS% 0x000567 0x906012 AG Set AG Star1 REF Position OK 
2024-03-20 05:01:10.0 CE TSC% > OCS% 0x000567 0x906012 AG Set AG Star1 REF Position COMPLETE%% 0006202403201501100
2024-03-20 05:01:19.8 CD OCS% > TSC% 0x000568 0x906012 AG Set AG Star1 REF Position 23182 17406
2024-03-20 05:01:1